## 필요 라이브러리 설치

In [6]:
# !pip install transformers
# !pip install googletrans==4.0.0-rc1

In [7]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel
from transformers import BlipProcessor, BlipForConditionalGeneration
from googletrans import Translator

## CLIP

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# 이미지를 로드합니다.
image = Image.open("/content/스크린샷 2024-11-11 163909.png")  # 이미지 경로를 지정하세요.

# 텍스트 설명을 정의합니다.
texts = ["a photo of hamburger", "a photo of pizza", "a photo of potato", "a person"]  # 분류할 텍스트 설명을 추가하세요.



In [10]:
# 이미지와 텍스트를 프로세서에 입력합니다.
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)

# 모델을 사용하여 logits를 계산합니다.
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # 이미지에 대한 logits
    probs = logits_per_image.softmax(dim=1)  # 확률로 변환

# 결과를 출력합니다.
for text, prob in zip(texts, probs[0]):
    print(f"{text}: {prob.item():.4f}")



a photo of hamburger: 0.9998
a photo of pizza: 0.0001
a photo of potato: 0.0001
a person: 0.0000


In [11]:
# 가장 높은 확률값과 해당 인덱스를 찾습니다.
max_prob, max_index = probs[0].max(dim=0)

# 결과를 출력합니다.
best_text = texts[max_index]
print(f"가장 높은 확률을 가진 텍스트: {best_text} (확률: {max_prob.item():.4f})")

가장 높은 확률을 가진 텍스트: a photo of hamburger (확률: 0.9998)


## 사진을 입력하면 알아서 사진에 대한 텍스트 설명을 생성해주는 모델

In [9]:
# 이미지 캡셔닝 모델과 프로세서를 초기화합니다.
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# # 이미지를 입력받아 텍스트 설명을 생성합니다.
# def generate_caption(image):
#     inputs = processor(images=image, return_tensors="pt")
#     out = caption_model.generate(**inputs)
#     caption = processor.decode(out[0], skip_special_tokens=True)
#     return caption

# # 이미지를 입력합니다.
# image = '/content/스크린샷 2024-11-11 163909.png'  # 여기에 이미지 데이터를 넣으세요.

# # 이미지에 대한 텍스트 설명을 생성합니다.
# text = generate_caption(image)

# # 텍스트 설명을 리스트로 변환합니다.
# texts = [text]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [10]:
# 이미지를 입력받아 텍스트 설명을 생성합니다.
def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt")
    out = caption_model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# 이미지를 입력합니다.
image_path = '.'  # 여기에 이미지 경로를 넣으세요.
image = Image.open(image_path)  # 이미지를 열어서 PIL.Image 형식으로 변환합니다.

# 이미지에 대한 텍스트 설명을 생성합니다.
text = generate_caption(image)

# 텍스트 설명을 리스트로 변환합니다.
texts = [text]

# 결과 출력
print(f"생성된 텍스트 설명: {text}")

# 영어 텍스트를 한국어로 번역합니다.
translator = Translator()
translated_text = translator.translate(text, dest='ko').text

# 한국어 결과 출력
print(f"한국어 번역: {translated_text}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


생성된 텍스트 설명: a bacon cheese burger with bacon and tomato
생성된 텍스트 설명: a bacon cheese burger with bacon and tomato
한국어 번역: 베이컨과 토마토를 곁들인 베이컨 치즈 버거
